In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
test = "mito_mem (3)"
print(test.split("(")[0].split(" ")[0])
for row in mi.rows:
    print(row.organelle_info)

In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
from annotator_metrics.src.preprocessing import copy_data
mi = MaskInformation()
print(mi.rows)
copy_data(group = "group1", output_base_path="/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/")

In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
print(mi.rows[0].organelle_info)


In [ ]:
import numpy as np
a=np.random.random((4,4,4))

In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
a=MaskInformation()
print(a.rows[1].organelle_info)

In [ ]:
from annotator_metrics.src.metrics_evaluations import calculate_all_to_all
a,b = calculate_all_to_all("group1","/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/", "Mean False Distance (Clipped)",num_workers = 10)

In [ ]:
from annotator_metrics.src.metrics_evaluations import compare_two_images
f1_score = compare_two_images(
    gt_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop171/jrc_mus-liver_crop171_ariadne_manual.tif',
    test_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop171/jrc_mus-liver_crop171_ariadne_refined.tif',
    organelle_label = 2,
    resolution = 8,
    metrics_to_calculate = "F1 Score")
print(f1_score)

f1_score = compare_two_images(
    gt_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop172/jrc_mus-liver_crop172_final_manual.tif',
    test_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop172/jrc_mus-liver_crop172_final_ariadne.tif',
    organelle_label = 2,
    resolution = 8,
    metrics_to_calculate = "F1 Score")
print(f1_score)


In [ ]:
from annotator_metrics.util.image_io import create_variance_images

create_variance_images("/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp","group1", "/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/n5s")

In [ ]:
import h5py
with h5py.File("/groups/cosem/cosem/annotations/amira//jrc_mus-liver/crop137/jrc_mus_liver_crop137_mito003.h5") as f:
    print(f["volumes"]["raw"].attrs["resolution"])
    print(f["volumes"]["labels"]["gt"].attrs["resolution"])



In [ ]:
1023/2

In [ ]:
{
  "dimensions": {
    "x": [
      resolution[0],
      "m"
    ],
    "y": [
      resolution[1],
      "m"
    ],
    "z": [
      resolution[2],
      "m"
    ]
  },
  "position": [
    dim[0]/2,
    dim[1]/2,
    dim[2]/2
  ],
  "crossSectionScale": 1,
  "projectionScale": 256,
  "layers": [
    {
      "type": "image",
      "source": "n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/03.n5/raw/",
      "tab": "source",
      "name": "raw"
    },
    {
      "type": "segmentation",
      "source": "n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/03.n5/a1/",
      "tab": "source",
      "name": "a1"
    }
  ],
  "selectedLayer": {
    "visible": true,
    "layer": "raw"
  },
  "layout": "4panel"
}


In [13]:
import neuroglancer
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/03.n5/raw')
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/03.n5/a1',
  source='n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/03.n5/b1')
print(neuroglancer.to_url(viewer.state).replace("https://","http://"))
neuroglancer.stop()

SyntaxError: keyword argument repeated (403307536.py, line 6)

In [1]:
from annotator_metrics.util.image_io import get_neuroglancer_view_of_crop
get_neuroglancer_view_of_crop(f"annotation_and_analytics/tmp/n5s/group1/10.n5")

http://neuroglancer-demo.appspot.com#!%7B%22layers%22:%5B%7B%22type%22:%22image%22,%22source%22:%5B%7B%22url%22:%22n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/10.n5/raw%22%7D%5D,%22name%22:%22raw%22%7D,%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/10.n5/gt%22%7D%5D,%22name%22:%22gt%22,%22visible%22:false%7D,%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/10.n5/a1%22%7D%5D,%22name%22:%22a1%22,%22visible%22:false%7D,%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/10.n5/a2%22%7D%5D,%22name%22:%22a2%22,%22visible%22:false%7D,%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22n5://http://10.150.100.248:8080/annotation_and_analytics/tmp/n5s/group1/10.n5/b1%22%7D%5D,%22name%22:%22b1%22,%

In [31]:
print(dirs)
print(dirs.sort(key=str))
print(dirs)

['a1', 'a2', 'b1', 'b2', 'c1', 'mito_lumen_variance', 'mito_mem_variance', 'mito_variance']
None
['a1', 'a2', 'b1', 'b2', 'c1', 'mito_lumen_variance', 'mito_mem_variance', 'mito_variance']
